# Load Packages

In [1]:
using Dates

include("printmat.jl")

printyellow (generic function with 1 method)

# Some Important Types

Julia has many different types of variables: signed integers (like 2 or -5), floating point numbers (2.0 and -5.1), bools (false/true), bitarrays (similar to bools, but with more efficient use of memory), strings ("hello"), Dates (2017-04-23) and many more. 

The numerical types also comes with subtypes for different precisions, for instance, Float16, Float32 and Float64. Unless you specify otherwise, code like
```
a = 2
b = 2.0
```
gives an Int64 and a Float64 respectively (at least on the 64 bit version of Julia).

## Integers and Floats

In [2]:
a = 2                   #integer, Int (Int64 on most machines)
b = 2.0                 #floating point, (Float64 on most machines)
A = [1;2]
B = [1.0;2.0]

println("a: ",typeof(a))
println(a)

println("\nb: ",typeof(b))
println(b)

println("\nA: ",typeof(A))
printmat(A)

println("B: ",typeof(B))
printmat(B)

a: Int64
2

b: Float64
2.0

A: Array{Int64,1}
         1
         2

B: Array{Float64,1}
     1.000
     2.000



## Why Use Int When There Are Floats? 

That is, why bother with sometimes using 3 when you could use 3.0 everywhere? Mostly because you cannot use 3.0 everywhere... 

For instance, you cannot pick out element `x[3.0]` from a vector. It has to be `x[3]`.

In [3]:
x = [1,10,100,1000]

#println(x[3.0])    #uncomment and run. Will give an error

println(x[3])

100


## Bools and BitArrays

In [4]:
c = 2 > 1.1
println("c: ",typeof(c))
println(c)

C = A .> 1.5
println("\nC: ",typeof(C))
printmat(C)

println("A BitArray is a more economical array version of Bool, but prints as 0/1 by printmat.\n", 
        "Notice that typeof(C[1]) gives: ",typeof(C[1]))

c: Bool
true

C: BitArray{1}
         0
         1

A BitArray is a more economical array version of Bool, but prints as 0/1 by printmat.
Notice that typeof(C[1]) gives: Bool


## Char and Strings

In [5]:
t = 'a'                                    #Char, just one letter
println(typeof(t))

txt = "Dogs are nicer than cats."          #String, could be a long novel 
println(typeof(txt))

Char
String


# Calculations with Mixed Types and Converting Types

A calculation like "integer" + "float" works and the type of the result is a float (the more flexible type). Similarly, "bool" + "integer" will give an integer. 

These promotion rules make it easy to have mixed types in calculations, and also provide a simple way of converting a variable from one type to another. There are also an explicit `convert()` function that might be quicker.

## Some Calculations with Mixed Types ("promotion")

In [6]:
println("integer + float: ",1+2.0)                #integer + float
println("bool + integer: ",(1 .> 0) + 2)          #bool + integer

integer + float: 3.0
bool + integer: 3


## Converting from Int to Float and Vice Versa

In [7]:
x = [1.1;10.1;100.1]
println("x: ",typeof(x))
printmat(x)

B_to_int = round.(Int,x)                     #float -> integer by rounding
println("rounding x to Int: ",typeof(B_to_int))
printmat(B_to_int)

println("A: ",typeof(A))
printmat(A)

A_to_float = A .+ 0.0                       #int -> float by adding 0.0
println("after converting A to Float64: ",typeof(A_to_float))   
printmat(A_to_float)                        #convert.(Float64,A) also works

x: Array{Float64,1}
     1.100
    10.100
   100.100

rounding x to Int: Array{Int64,1}
         1
        10
       100

A: Array{Int64,1}
         1
         2

after converting A to Float64: Array{Float64,1}
     1.000
     2.000



## Converting from Bools and BitArrays to Int and Vice Versa

In [8]:
C_to_int = C .+ 0                        #BitArray -> int by adding 0
println(typeof(C_to_int))                #convert.(Int,C) also works
printmat(C_to_int)

D = [1;0;1]
D_to_bit = D .> 0                        #int -> BitArray by comparing
println(typeof(D_to_bit))                #convert(BitArray,D) also works 
printmat(D_to_bit)

Array{Int64,1}
         0
         1

BitArray{1}
         1
         0
         1



## From Bools and BitArrays to Int: A Tricky Case (extra)

`false` is a "strong zero" in the sense that 
`false*NaN == 0` and `false*Inf == 0`. 

If you do not want that behaviour in your code, transform `false` to 0 and then multiply.

In [9]:
println(false*NaN)
println(false*Inf)

println(convert(Int,false)*NaN)
println(convert(Int,false)*Inf)

0.0
0.0
NaN
NaN


# Testing the Type

The perhaps easiest way to test the type is by using the `isa(variable,Type)` function.

Notice that an array has the type `Array` and more specifically, `Array{Float64}` if it is an array with Float64 numbers.

In [10]:
x = 1.2
z = [1.2,1.3]

println("$x is a Number: ",isa(x,Number))
println("$x is an Int: ",isa(x,Int))
println("$x is an Int or a Float64: ",isa(x,Union{Int,Float64}))

println("$z is a Float64: ",isa(z,Float64))
println("$z is an Array: ",isa(z,Array))

1.2 is a Number: true
1.2 is an Int: false
1.2 is an Int or a Float64: true
[1.2, 1.3] is a Float64: false
[1.2, 1.3] is an Array: true


# Type Instability

Your code will run faster if your variables do not change type in the computations. The next cells illustrate that.

In [11]:
function fn1(n)
    x = 0                    #x starts out as an Int
    for i = 1:n
        x = x + 0.1          #x is changed from Int to a Float when you add 0.1
    end
    return x
end

function fn2(n)
    x = 0.0                  #x starts out as a Float
    for i = 1:n
        x = x + 0.1          #x remains a float
    end
    return x
end

fn2 (generic function with 1 method)

In [12]:
x = fn1(10)                #a "dry" run makes the subsequent timing results more accurate
x = fn2(10)

@time fn1(1e+6)
@time fn2(1e+6)

println("\nfn2() is much faster and uses much less memory (even a more serious timing would show that...)")

  0.090474 seconds (305.01 k allocations: 15.666 MiB)
  0.007907 seconds (17.16 k allocations: 901.522 KiB)

fn2() is much faster and uses much less memory (even a more serious timing would show that...)


In [13]:
@code_warntype fn1(1e+6)   #to spot type instability, run this. Notice the red warnings.

#@code_warntype fn2(1e+6)  #compare with this

Variables
  #self#::Core.Compiler.Const(fn1, false)
  n::Float64
  x::Union{Float64, Int64}
  @_4::Union{Nothing, Tuple{Float64,Int64}}
  i::Float64

Body::Union{Float64, Int64}
1 ─       (x = 0)
│   %2  = (1:n)::StepRangeLen{Float64,Base.TwicePrecision{Float64},Base.TwicePrecision{Float64}}
│         (@_4 = Base.iterate(%2))
│   %4  = (@_4 === nothing)::Bool
│   %5  = Base.not_int(%4)::Bool
└──       goto #4 if not %5
2 ┄ %7  = @_4::Tuple{Float64,Int64}::Tuple{Float64,Int64}
│         (i = Core.getfield(%7, 1))
│   %9  = Core.getfield(%7, 2)::Int64
│         (x = x + 0.1)
│         (@_4 = Base.iterate(%2, %9))
│   %12 = (@_4 === nothing)::Bool
│   %13 = Base.not_int(%12)::Bool
└──       goto #4 if not %13
3 ─       goto #2
4 ┄       return x
